# 什么是语音识别？

建模条件概率，假设文本是Y，语音是X。那么模型是$$W^* = argmax_{W}P(Y|X)$$
公式的含义是，在给定语音X的条件下，使条件概率最大的文本$W^*$即目标文本。

通过贝叶斯定理，条件概率可以转换为$$P(X|Y)P(Y)$$
其中P(X|Y)就是通过语音与对应文本训练出来的**声学模型**，其中Y可以认为是语言学家研究出来的关于语音文本表示的先验知识，X是在这种先验知识上建立的对语音的分布。而P(Y)可以认为是**语言模型**。  

## 声学模型与语言模型的工作分别说是什么？
声学模型描述的是，在给定先验词序列或因素序列Y信息时，观察到语音X的后验概率，作用时建模语音信号与基本词、音素之间的映射关系。语言模型学习序列的上下文关系（长期依赖关系），来生成更加符合逻辑的文本。
___

## 语音识别的发展：
在深度学习技术崛起之前，在语音识别领域较为优秀的系统使GMM-HMM混合模型，GMM高斯混合模型负责映射语音帧到语音识别的基本音素，HMM隐马尔可夫模型描述了各个音素token之间的转移关系。  
在深度学习崛起之后，诞生了DNN-GMM混合模型。  
但是，这些方法都需要大量的语言学知识，开发与维护成本很高，近年来，随着语言大模型的高速发展，不需要任何语言学知识的端到端语言识别系统成为了技术的焦点。  
端到端语言识别系统的建模方式大致分为两种：**CTC序列标注**、**AED基于注意力机制的自回归序列生成**。

___

# CTC建模是什么？解决了什么问题？
CTC对编码后的语音帧进行逐帧标注，每一帧都会输出对应的token。  
在理想语音识别中，每一帧的结果已知，我们只需要在这种已知的对应关系下，最大化条件概率就好了。
但在现实应用中，帧对齐是未知的，因此我们只能穷举所有的帧对齐可能，最终的优化目标就是多个对齐可能条件概率的加和。

## 如何实现对CTC所有对齐的穷举？
使用深度搜索算法（ppt中硕士动态规划），在以帧为列、词为行的矩阵中，找到所有从左上角到右下角的路径。

## CTC的反向传播计算？
+ 直接使用pytorch自动求导
+ 直接计算求导公式

## CTC的解码问题？
CTC在解码时，可能会遇到叠词被合并为一个词的可能，因此需要在穷举可能中，插入Blank，使得叠词被区分。  
此外，CTC解码有多种方法：贪心法、beam search方法。
+ 贪心算法：对每一帧输出的概率分布，我们直接拿每一帧概率最高的token。由于每一帧标记都是独立的，因此贪心算法得到的解码结果就是最大可能性结果。
+ beam search：在第一帧输出中，找到N个最高的概率，接着往之后的帧连接，每次都抽取概率最高的N个路径。在CTC中，beam search得到的解码结果实际上和贪心的结果是一样的。

___

# AED基于注意力机制的自回归序列生成方法。
AED方法基于循环神经网络、transformer、TCN等时序神经网络模型，基于语音编码结果，输出文本序列。类似于语言模型的解码器。AED是**自回归的**，因此每一个token的输出都不是独立的，依赖于之前的输入。  
AED基于注意力机制，实现了输出与输入帧之间的软对齐，每一个token的输入，只基于输入序列的部分信息。AED输出的文本以sos开始，以eos结束。  
常用损失函数：**交叉熵损失**

## AED常用的结构：
+ RNN：非常经典的自回归神经网络，对于每一个时间步的输出共享权重W，b。
+ TCN：沿着时间序列的一维卷积，不同时间步，会共享一个卷积核。
+ transformer：自注意力层由于每个时间步的输入权重都取决于对其他时间步输入的注意力，因此每一个时间步的权重都不一致。

___


# 问答汇总

## 传统GMM-HMM成本分析
+ 需要对词序列、音素序列与语音帧进行强制帧对齐，只有提前进行强制帧对齐，才能对混合模型进行有效训练。
+ 需要大量的语言学专家知识。

## 什么是流式解码？为什么原始的AED模型不能进行流式解码？如何改进AED模型以支持流式解码？
流式解码即输入当前语音帧，会实时地对当前语音帧进行语音识别，而不需要等到整个序列输入完毕。原始的AED的注意力机制会依赖于整个序列，因此不能进行流式解码。
改进的策略：
+ 使用因果掩码，让注意力机制只能关注过去与现在的信息。
+ 使用分块处理，对输入序列进行分块，让注意力机制在每个编码器分块中学习上下文，而无需等待全部序列输入完毕。

## 为何CTC可以使用贪心算法就可以找到最优解码，而AED不行。
CTC的每个token输出的概率分布是独立的，因此取每一帧的最高概率得到的乘积，就是当前对齐模式下的最大概率了（虽然不是全局最大概率）。AED每一个token的概率分布是条件概率，依赖于前面所有tokens的概率分布，因此不能直接简单相乘得到最优解码。

___

# Conformer
## Swish activation
y = x * sigmoid(x)  
特点：无上界有下届，平滑非单调，处处可导。

## 深度可分离卷积
Conformer中的卷积层内部实际上是深度可分离卷积，分别由逐点卷积-swish激活函数-GLU门控线性单元-深度卷积-swish激活函数-深度卷积-dropout组成。  
深度可分离卷积一般由逐点卷积和深度卷积共同构成，由于传统的卷积核参数量比较大，因此在参数量有限的情况下，希望完成卷积的功能，可以选择**深度可分离卷积**。
### 逐点卷积  
卷积核大小为1*1，卷积方式和常规的卷积一致，用于融合多通道的信息，改变通道数。  
![](2.jpeg)
### 深度卷积
深度卷积的运行模式不同于常规卷积，卷积核的通道数与输入通道数一致，且每个卷积核只与对应的通道卷积。深度卷积改变每一个通道的维度，但是不改变通道数量。  
深度卷积只提取当前通道的局部特征，而不能提取多通道之间的特征。
![](1.jpeg)  

### Conformer的整体结构
conformer的灵感来源于transformer和macaron-net，半步长前馈-多头注意力机制-深度可分离卷积-半步长前馈-dropout。


___

## ctc的三种解码方式
+ 贪心算法：取每一帧的最大概率token，直接解码token序列
+ beam search方法：用动态规划的方法，维护n个最高概率的token序列，对于每一个序列，都与下一帧的所有概率相乘，寻找最高的概率作为该序列的新值。
+ prefix beam search方法：由于beam search只维护n个最高可能性的token序列，而这些token序列可能会解码为同一个文本序列，由于beam search方法并不能在计算过程中合并这些冗余的token序列，因此会造成造成**计算的冗余**。此外beam search本身得到的是带重复字符和空格的token序列，还需要进一步解码才能得到最终的文本序列。由于beam search在每个时间步只维护概率最高的n个token序列，这使得beam search会忽略掉一些概率更高的token序列。因此引入prefix beam search，它不再维护token序列，而是直接维护最终输出文本序列的n个最大概率前缀字符串。这样可以避免冗余计算，找到全局概率最高的n个输出文本，且无需进一步解码。

___